<a href="https://colab.research.google.com/github/miller00315/crew_ai_studies/blob/main/financial_analisys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install crewai crewai_tools langchain_community

In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [3]:
import os

os.environ["SERPER_API_KEY"] = ''
os.environ["GROQ_API_KEY"] = ''

In [4]:
from crewai import Agent, Crew, Task, LLM

In [5]:
llm = LLM(model="groq/mixtral-8x7b-32768", api_key=os.environ["GROQ_API_KEY"])

# **crewAI Tools**

In [6]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_config.py:295: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/usr/local/lib/python3.11/dist-packages/crewai_tools/tools/scrapegraph_scrape_tool/scrapegraph_scrape_tool.py:34: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  @validator("website_url")
/usr/local/lib/python3.11/dist-packages/crewai_tools/tools/selenium_scraping_tool/selenium_scraping_tool.py:26: PydanticDeprecatedSince20: Pydantic V1 style `@validator`

In [7]:
data_analyst_agent = Agent(
    llm=llm,
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [8]:
trading_strategy_agent = Agent(
    llm=llm,
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [9]:
execution_agent = Agent(
    llm=llm,
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [10]:
risk_management_agent = Agent(
    llm=llm,
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

**Creating Tasks**

In [11]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [12]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)

In [13]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [14]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

**Creating the Crew**
* The Process class helps to delegate the workflow to the Agents (kind of like a Manager at work)
* In the example below, it will run this hierarchically.
manager_llm lets you choose the "manager" LLM you want to use.

In [15]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent,
            trading_strategy_agent,
            execution_agent,
            risk_management_agent],

    tasks=[data_analysis_task,
           strategy_development_task,
           execution_planning_task,
           risk_assessment_task],

    manager_llm=llm,
    process=Process.hierarchical,
    verbose=True
)

**Running the Crew**
* Set the inputs for the execution of the crew.

In [16]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

In [17]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 
[2025-03-12 01:56:45][🚀 CREW 'CREW' STARTED, DD725C56-24AE-49AA-9C7F-EB68033E3019]: 2025-03-12 01:56:45.177349
 
[2025-03-12 01:56:45][📋 TASK STARTED: CONTINUOUSLY MONITOR AND ANALYZE MARKET DATA FOR THE SELECTED STOCK (AAPL). USE STATISTICAL MODELING AND MACHINE LEARNING TO IDENTIFY TRENDS AND PREDICT MARKET MOVEMENTS.]: 2025-03-12 01:56:45.197591
 
[2025-03-12 01:56:45][🤖 AGENT 'CREW MANAGER' STARTED TASK]: 2025-03-12 01:56:45.199015
# Agent: Crew Manager
## Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.
 
[2025-03-12 01:56:45][🤖 LLM CALL STARTED]: 2025-03-12 01:56:45.199234
 
[2025-03-12 01:56:46][✅ LLM CALL COMPLETED]: 2025-03-12 01:56:46.932445
 
[2025-03-12 01:56:46][🤖 TOOL USAGE STARTED: 'READ WEBSITE CONTENT']: 2025-03-12 01:56:46.933608
 
[2025-03-12 01:56:48][✅ TOOL USAGE FINISHED: 'READ WEBSITE CONTENT']: 2025-03-12 01:56:48.299311


# Agent: Cr

ERROR:root:LiteLLM call failed: litellm.RateLimitError: RateLimitError: GroqException - {"error":{"message":"Request too large for model `mixtral-8x7b-32768` in organization `org_01jp410k5keg580gff4q5h8a22` service tier `on_demand` on tokens per minute (TPM): Limit 5000, Requested 7954, please reduce your message size and try again. Visit https://console.groq.com/docs/rate-limits for more information.","type":"tokens","code":"rate_limit_exceeded"}}





LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.


Provider List: https://docs.litellm.ai/docs/providers

 
[2025-03-12 01:56:54][❌ LLM CALL FAILED: 'LITELLM.RATELIMITERROR: RATELIMITERROR: GROQEXCEPTION - {"ERROR":{"MESSAGE":"REQUEST TOO LARGE FOR MODEL `MIXTRAL-8X7B-32768` IN ORGANIZATION `ORG_01JP410K5KEG580GFF4Q5H8A22` SERVICE TIER `ON_DEMAND` ON TOKENS PER MINUTE (TPM): LIMIT 5000, REQUESTED 7954, PLEASE REDUCE YOUR MESSAGE SIZE AND TRY AGAIN. VISIT HTTPS://CONSOLE.GROQ.COM/DOCS/RATE-LIMITS FOR MORE INFORMATION.","TYPE":"TOKENS","CODE":"RATE_LIMIT_EXCEEDED"}}
']: 2025-03-12 01:56:54.672779
 Error during LLM call: litellm.RateLimitError: RateLimitError: GroqException - {"error":{"message":"Request too large for model `mixtral-8x7b-32768` in organization `org_01jp410k5keg580gff4q5h8a22` service tier `on_demand` on tokens per minute (TPM): Limit 5000, Requested 7954, please reduce your message size and try again. Visit https://console.groq.com/docs/rat

RateLimitError: litellm.RateLimitError: RateLimitError: GroqException - {"error":{"message":"Request too large for model `mixtral-8x7b-32768` in organization `org_01jp410k5keg580gff4q5h8a22` service tier `on_demand` on tokens per minute (TPM): Limit 5000, Requested 7954, please reduce your message size and try again. Visit https://console.groq.com/docs/rate-limits for more information.","type":"tokens","code":"rate_limit_exceeded"}}


In [ ]:
from IPython.display import Markdown
Markdown(str(result))